In [1]:
! pip install -U datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
! pip install -U sentence_transformers -q

In [3]:
from sentence_transformers import SentenceTransformer,losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datasets import load_dataset
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from sentence_transformers.trainer import SentenceTransformerTrainer
import torch

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
train_ds=load_dataset("glue","mnli",split="train").select(range(50000))
train_ds=train_ds.remove_columns("idx")
train_ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 50000
})

In [6]:
train_ds.to_pandas().head()

,premise,hypothesis,label
0,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,1
1,you know during the season and i guess at at y...,You lose the things to the following level if ...,0
2,One of our number will carry out your instruct...,A member of my team will execute your orders w...,0
3,How do you know? All this is their information...,This information belongs to them.,0
4,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,1


In [7]:
# create evaluator
#1. load eval dataset
evaluation_ds=load_dataset("glue","stsb",split="validation").remove_columns("idx")

train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [8]:
evaluation_ds.to_pandas().tail(5)

,sentence1,sentence2,label
1495,Scientists prove there is water on Mars,Has Nasa discovered water on Mars?,2.0
1496,Pranab stresses need to strive for peace by na...,WTO: India regrets action of developed nations,0.0
1497,Volkswagen skids into red in wake of pollution...,"Volkswagen's ""gesture of goodwill"" to diesel o...",2.0
1498,Obama is right: Africa deserves better leadership,Obama waiting for midterm to name attorney gen...,0.0
1499,New video shows US police officers beating men...,New York police officer critically wounded in ...,0.0


In [9]:
#2. create evaluator
evaluator=EmbeddingSimilarityEvaluator(
    sentences1=evaluation_ds["sentence1"],
    sentences2=evaluation_ds["sentence2"],
    scores=[label/5 for label in evaluation_ds["label"]],
    main_similarity="cosine"
)

In [10]:
# load our model
model_name="sentence-transformers/all-MiniLM-L6-v2"
device="cuda" if torch.cuda.is_available() else "cpu"
model=SentenceTransformer(model_name,device=device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# try to tokenize a sentence
ids=torch.tensor(model.tokenizer.encode("hello my name abdeljalil"))
print(ids)
model.tokenizer.convert_ids_to_tokens(ids)

tensor([  101,  7592,  2026,  2171, 19935,  2884, 28948,  2140,   102])


['[CLS]', 'hello', 'my', 'name', 'abd', '##el', '##jali', '##l', '[SEP]']

In [12]:
# test model
model.encode("hello abdeljalil").shape

(384,)

In [13]:
# create a loss function
train_loss=losses.MultipleNegativesRankingLoss(model=model)

In [14]:
# arguments hp
args=SentenceTransformerTrainingArguments(
    output_dir="ft_st_all_mini_lm_round2",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    fp16=True,
    eval_steps=100,
    logging_steps=100
)

In [15]:
from sentence_transformers.trainer import SentenceTransformerTrainer
trainer=SentenceTransformerTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=evaluation_ds,
    args=args,
    evaluator=evaluator,
    loss=train_loss
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [16]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


dataset = dataset.select_columns(['hypothesis', 'entailment', 'contradiction'])


Step,Training Loss
100,0.110500
200,0.089900
300,0.060500
400,0.084100
500,0.072700
600,0.090200
700,0.068700
800,0.093900
900,0.053500
1000,0.084800


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=3125, training_loss=0.07182910659790039, metrics={'train_runtime': 269.0373, 'train_samples_per_second': 185.848, 'train_steps_per_second': 11.615, 'total_flos': 0.0, 'train_loss': 0.07182910659790039, 'epoch': 1.0})

In [18]:
evaluator(model)

{'pearson_cosine': 0.8409014205648969,
 'spearman_cosine': 0.841783975800472,
 'pearson_manhattan': 0.8456959766644842,
 'spearman_manhattan': 0.8411468153598396,
 'pearson_euclidean': 0.8463347024709539,
 'spearman_euclidean': 0.841783975800472,
 'pearson_dot': 0.840901422990466,
 'spearman_dot': 0.8417838371517684,
 'pearson_max': 0.8463347024709539,
 'spearman_max': 0.841783975800472}